## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv("GROQ_API_KEY")
groq_api_key

'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'

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000245A024D720>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000245A024E3E0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [8]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is ali and I am a LLM Engineer")])

AIMessage(content="Hello Ali, it's nice to meet you!  \n\nThat's fascinating work you do as an LLM Engineer.  Is there anything in particular you're working on that you'd like to talk about? I'm always eager to learn more about the advancements in language models.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 64, 'prompt_tokens': 22, 'total_tokens': 86, 'completion_time': 0.116363636, 'prompt_time': 0.00024848, 'queue_time': 0.0141354, 'total_time': 0.116612116}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-2464bda8-d3b1-4a82-ab13-94a754a275c8-0', usage_metadata={'input_tokens': 22, 'output_tokens': 64, 'total_tokens': 86})

In [9]:
from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi , My name is ali and I am a LLM Engineer"),
        AIMessage(content="Hi ali! 🤖\n\nIt's nice to meet you. That's a really cool job! What kind of projects are you working on as an LLM Engineer?  \n\nI'm always interested to learn more about how people are using large language models. 😊\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
)

AIMessage(content="You're Ali, and you're an LLM Engineer!  \n\nThat's what you told me earlier. 😊  Is there anything else you'd like to tell me about yourself or your work? \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 101, 'total_tokens': 149, 'completion_time': 0.087272727, 'prompt_time': 0.003868896, 'queue_time': 0.009823552, 'total_time': 0.091141623}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dd1986bd-7060-4418-b6de-c8995783ca47-0', usage_metadata={'input_tokens': 101, 'output_tokens': 48, 'total_tokens': 149})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [6]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [7]:
config={"configurable":{"session_id":"chat1"}}

In [11]:
response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is ali and I am a LLM Engineer")],
    config=config
)
response.content

"Hi Ali! It's great to meet you.  \n\nBeing an LLM Engineer is such a cutting-edge field. What kinds of projects are you working on? Are you focused on research, development, or deployment of these powerful models?  \n\nI'm always curious to hear about the latest advancements in the world of AI. 😊  \n\n"

In [12]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Ali! 😊  \n\nI remember that from our first introduction.  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 198, 'total_tokens': 228, 'completion_time': 0.054545455, 'prompt_time': 0.007178015, 'queue_time': 0.006293263999999999, 'total_time': 0.06172347}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-dba42449-47ce-41b8-9a5f-4c28d1d7008c-0', usage_metadata={'input_tokens': 198, 'output_tokens': 30, 'total_tokens': 228})

In [13]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to know!\n"

In [14]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you!  Is there anything I can help you with today?\n"

In [15]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, you told me a little while ago! 😊  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [16]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [17]:
chain.invoke(
    {"messages":[HumanMessage(content="Hi My name is ali")]}
)

AIMessage(content="Hi Ali! Nice to meet you.\n\nAsk me anything! I'll do my best to answer your questions. 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 31, 'total_tokens': 60, 'completion_time': 0.052727273, 'prompt_time': 0.000311429, 'queue_time': 0.013461731, 'total_time': 0.053038702}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-6e280ab2-68f9-431c-8bc5-bfa2ce0a823a-0', usage_metadata={'input_tokens': 31, 'output_tokens': 29, 'total_tokens': 60})

In [ ]:
# To record history
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [19]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is ali")],
    config=config
)
response

AIMessage(content="Hi Ali, it's nice to meet you! \n\nI'm happy to help answer your questions to the best of my ability.  \n\nWhat can I do for you today? 😊 \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 31, 'total_tokens': 75, 'completion_time': 0.08, 'prompt_time': 0.000309349, 'queue_time': 0.0143208, 'total_time': 0.080309349}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-e6735c8c-11ba-43ab-9d72-f066d44b3578-0', usage_metadata={'input_tokens': 31, 'output_tokens': 44, 'total_tokens': 75})

In [22]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Ali.  I told you just now!  \n\nIs there anything else I can help you with? 😊 \n'

In [23]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [26]:
response=chain.invoke({"messages":[HumanMessage(content="assalamu alaikum My name is ali")],"language":"Hindi"})
response.content

'Wa alaikum assalam Ali! \n\nमैं आपकी मदद करने के लिए तैयार हूँ। आपके कोई सवाल हैं या मैं आपके लिए कुछ कर सकता हूँ? \n\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [27]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [34]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="assalam u alaikum,I am ali")],"language":"Hindi"},
    config=config
)
repsonse.content

'وعلیکم السلام، अली भाई! 😊\n\nआप कैसे हैं? मुझे बताइए, मैं आपकी मदद कैसे कर सकता हूँ?  \n'

In [35]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)
response.content

'आपका नाम अली है।  😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [38]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

e:\Project Commit\Gen AI\Building_chatbot_with_coversational_history\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
e:\Project Commit\Gen AI\Building_chatbot_with_coversational_history\venv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Insha sadaf\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For mor

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [39]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As a large language model, I don't have access to your personal preferences like your favorite ice cream flavor.  \n\nWhat's your favorite ice cream flavor?  🍦😊\n"

In [40]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2". 😊  \n'

In [41]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [42]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I have no memory of past conversations and don't know your name.  \n\nWhat's your name? 😊  \n"

In [43]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a helpful assistant with no memory of past conversations, I have no recollection of any math problem you may have asked. \n\nIf you'd like to ask me a math problem now, I'd be happy to help! 😊  \n\n"